Challenge Begins
---

As time flies, Now the time let us attend compettiion:

[Tbrain by Trend-Micro](https://tbrain.trendmicro.com.tw)  台灣 ETF 價格預測競賽

<big>
    
|序號 | 欄位 | 說明       |
| ------| :------: | :------: |
|0|ETFid|ETF 代號|
| 1| Mon_ud | 周一漲跌 (1:漲, 0:平, -1:跌) 50% |
| 2| Mon_cprice| 周一收盤價 (權重 50%) |
| 3| Tue_ud | 周二漲跌 (1:漲, 0:平, -1:跌) 50% |
| 4| Tue_cprice| 周二收盤價 (權重 50%) |
| 5| Wed_ud | 周三漲跌 (1:漲, 0:平, -1:跌) 50% |
| 6| Wed_cprice| 周三收盤價 (權重 50%) |
| 7| Thu_ud | 周四漲跌 (1:漲, 0:平, -1:跌) 50% |
| 8| Thu_cprice| 周四收盤價 (權重 50%) |
| 9| Fri_ud | 周五漲跌 (1:漲, 0:平, -1:跌) 50% |
| 10| Fri_cprice| 周五收盤價 (權重 50%) |


Sample

```
ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0050,1,87.15,1,88.05,-1,88.00,-1,87.25,1,87.45
...
```

In [ ]:
from hmmlearn.hmm import GaussianHMM
import numpy as np
from matplotlib import cm, pyplot as plt
import matplotlib.dates as dates
import pandas as pd
import datetime

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import pandas as pd
import numpy as np

from fbprophet import Prophet
import pandas_profiling as pp
#from matplotlib import cm, pyplot as plt
import matplotlib.dates as dates
#import pandas_datareader.data as web
from pandas_datareader import data as web
import datetime
from hmmlearn.hmm import GaussianHMM

import fix_yahoo_finance as yf
yf.pdr_override()

In [ ]:
import time
today=time.strftime("%Y-%m-%d")
today

In [ ]:
n = 6 

ETF1 = "0050.TW"

END_DATE=today
START_DATE = '2013-01-02'



In [ ]:
df_ETF1 = web.get_data_yahoo(ETF1,START_DATE,END_DATE)
df_ETF1

In [ ]:
df_ETF1.info()

In [ ]:
file="TBrain_Round2_DataSet_20180331/taetfp-utf.csv"
df=pd.read_csv(file)

In [ ]:
df

In [ ]:
df[1870:1871]

**First problem to fixed**

Volumes more than 1000 should be in the form, **xx,xxx**, and  other less than 1000, **xxx**. This caused the problem while load csv data: the forth in two columns and the latter only one column. 

To solve the problem, **1.** create a new column, named "total", which is the sum of first column,  multiplyed in thousand and the second column. But the data with volumes less than 1000 is calculated as `NaN` since the scecond column is `NaN`. Thus, **2.** duplicate the volume in the first column to the "total" column for the data less than 1000.      

In [ ]:
# create the new column, total for volumes more than 1000
df.total=df['成交張數(張)1000']*1000+df['成交張數(張)']

In [ ]:
df['total'] = df.apply(lambda row: row['成交張數(張)1000']*1000+row['成交張數(張)'], axis=1)

In [ ]:
# duplicate the first volume to tatal column if volume is less than 1000
df['total'].fillna(df['成交張數(張)1000'], inplace=True)

In [ ]:
# View the data of 0050 ETFcch
df[df['代碼']==50]

In [ ]:
df.info()

Secondly, conversion of datetime: **1.** convert `YYYYMMDD` to `YYYY-MM-DD` and put in new column 'Date'; **2.** create anew column, weekday, for each deal day. 

In [ ]:
df['Date']= df.apply(lambda row:pd.to_datetime(str(row['日期']), format='%Y%m%d') , axis=1)

In [ ]:
df['weekday']=pd.DatetimeIndex(df.Date).dayofweek.values

In [ ]:
df.head()

In [ ]:
# Make a copy after first data cleaning
df1=df[['代碼', '日期', '中文簡稱', '開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', 'total','Date','weekday']]
df1.to_csv("TBrain_Round2_DataSet_20180331/taetfp-utf-1.csv",index=False)

In [ ]:
set(df['中文簡稱'])cch

|code|ETF|code|ETF|
| ------| :------: | :------: |---|
|0050|元大台灣50|0058|富邦發達|
|0051|元大中型100 |0059|富邦金融|
|0052|富邦科技|006201|元大富櫃50|
|0053|元大電子|006203|元大MSCI台灣|
|0054|元大台商50|006204|永豐臺灣加權 |
|0055|元大MSCI金融|006208|富邦台50|
|0056|元大高股息|00690|兆豐藍籌30|
|0057|富邦摩台|00692|富邦摩台|
|00701|國泰臺灣低波動30|00713|元大台灣高息低波|

In [ ]:
# manually define the name of ETF's
etfs=['0050','0051','0052','0053','0054','0055','0056','0057','0058',
      '0059','006201','006203','006204','006208','00690','00692','00701','00702']

In [ ]:
import os, syscch

In [ ]:
if os.path.exists("data"):
   print("exist")
else:
   os.mkdir("data") 

In [ ]:
# Seperate the data by ETF's and save them indivually into sub-folder, data, in the name of ETF number 
for etf in etfs:
    dfname=('df%s' %etf)
    filename=('data/%s.csv' %etf)
    ind=int(etf[2:])

    dfname=df1[df1['代碼']==ind] 
    dfname.to_csv(filename,index=False)

Prediction
---
1. load csv, data/0050.csv
- data cleaning:
  - index with 'Date'
  - convert index into valid datetime formatcch
  - translate the columns into English
- Simple prediction
  - Linear Regrssion by scikit-learn

In [ ]:
df0050=pd.read_csv("data/0050.csv")

In [ ]:
# note: Date is in object format not in datetime
df0050.info()

In [ ]:
df0050_date=pd.read_csv("data/0050.csv",index_col='Date')

In [ ]:
df0050_date.head()

In [ ]:
# no valid datetime index, no datetime X-axis
df0050_date['收盤價(元)'].plot()

In [ ]:
# convert index to valid datetime format
df0050_date.index=pd.to_datetime(df0050_date.index)

In [ ]:
# et voilá
df0050_date['收盤價(元)'].plot(figsize=[12,6])

In [ ]:
df0050_date.columns.values

In [ ]:
df0050_date=df0050_date.rename(columns={"代碼": "code", "日期": "date",'中文簡稱':'Name',"開盤價(元)":'Open',
                            "最高價(元)":'High',"最低價(元)":'Low',"收盤價(元)":'Close'});


In [ ]:
df0050_date.head()

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Create linear regression object
LinReg = linear_model.LinearRegression()

In [ ]:
X=(df0050_date.index - df0050_date.index[0]).days.values.reshape(-1, 1)
Y=df0050_date.Close

# Train the model using the training sets
LinReg.fit(X,Y)

In [ ]:
(df0050_date.index - df0050_date.index[0]).days.values

In [ ]:
# Make predictions using the testing set
Y_pred = LinReg.predict(X)

In [ ]:
df0050_date['pred']=Y_pred 

In [ ]:
df0050_date

In [ ]:
# The coefficients
print('Prediction: \n', LinReg.intercept_,' + ',LinReg.coef_,'X_Close')
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y, Y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y, Y_pred))


In [ ]:
# Finally
df0050_date['Close'].plot(figsize=[12,6])
df0050_date['pred'].plot()

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(df0050_date.index, Y,  color='r', alpha=0.3)
plt.plot(df0050_date.index, Y_pred, color='blue', linewidth=2)

#plt.xticks(())
#plt.yticks(())